In [ ]:
import serial                            # imports the 'pyserial' library. This library allows Python to communicate with the Serial Ports
import pandas as pd                      # imports the 'pandas' library which is used for handling data tables (CSV in our case)
import numpy as np                       # imports the 'numpy' library which is used for fast mathematical operations
import re                                # imports the 're' (Regular Expression) library. We use it to find patterns (numbers in our case) inside text 
from collections import deque            # imports the 'deque' (double-ended queue) data structure to use as an optimized list for buffers
import time                              # imports the 'time' library which provides time-related functions

# imports specific tool from 'sklearn' (Scikit-Learn) Machine Learning library
from sklearn.model_selection import train_test_split            # tool to split data into 'training' and 'testing' sets
from sklearn.ensemble import RandomForestClassifier             # the ML model we are using (a collection of silly decision trees)
from sklearn.metrics import accuracy_score                      # tools to grade the models performance
from sklearn.decomposition import PCA                           # tool to help reduce the size of the dataset
from sklearn.impute import SimpleImputer                        # tool to replace missing data with plausible values
from scipy.signal import savgol_filter                          # tool to filter noise and smooth data

# Configuration
COM_PORT = 'COM4'                             # the COM port where the RX is connected
BAUD_RATE = 921600                            # baud rate for serial communication (matches the RX configuration)
training_file = "filtered_csi_dataset.csv"    # the name of the file used to train the model before seeing it's capabilities in real-time

print(f"Loading and processing the dataset...")

try:
    # reads the CSV file into a DataFrame (a table structure)
    # header = None -> tells pandas there are no column names in the first row
    # on_bad_lines = 'skip' -> ignores corrupted rows instead of crashing the program
    # engine = 'python' -> uses the Python parsing engine
    df = pd.read_csv(training_file, header = None, on_bad_lines = 'skip', engine = 'python')

    # standard error handling
except FileNotFoundError:
    print(f"Error: file {training_file} not found.")
    raise   # stops the program if the file is missing

# y_raw contains the 'answers' (0 for empty, 1 for person in room)
y_raw = df.iloc[:,0].values

# X_raw contains the 'questions' (the raw CSI data)
X_raw = df.iloc[:,4:].values

# the raw data is interleaved (real, imaginary, real, imaginary...)
# we extract the real parts by taking every 2nd column starting from index 0
# we extract the imaginary parts by taking every 2nd column starting from index 1
real_parts = X_raw[:, 0::2]
imaginary_parts = X_raw[:, 1::2]

# we calculate the amplitude (signal strength) using Pythagorean theorem
# sqrt(real^2 + imaginary^2)
X_amp = np.sqrt(real_parts**2 + imaginary_parts**2)

# handles missing values (NaNs) caused by errors by replacing them with thea mean of the column
imputer = SimpleImputer(strategy = 'mean')

# apply imputation to our data
X_amp = imputer.fit_transform(X_amp)

# apply Savitzky-Golay filter across time (axis = 0 -> rows), not across subcarriers
X_amp = savgol_filter(X_amp, window_length = 11, polyorder = 3, axis = 0)

# calculate the average signal strength for each column (axis = 0) across the entire dataset
X_mean = np.mean(X_amp, axis = 0)

# calculate the standard deviation to measure how much the signal usually fluctuates
X_std = np.std(X_amp, axis = 0)

# apply Z-score standardization -> center data around 0 and scale it (add a small epsilon to avoid division by 0)
X_amp = (X_amp - X_mean) / (X_std + 0.00001)

# keep 95% information
pca = PCA(n_components = 0.95)

# reduce the dimension of the data
X_amp = pca.fit_transform(X_amp)

# calculate the absolute difference between the current row and the previous row to detect motion
# prepend = X_amp[0:1] ensures the result has the same number of rows as the input
delta = np.abs(np.diff(X_amp, axis = 0, prepend = X_amp[0:1]))

# rolling standard deviation for small movements detection
# we calculate the standard deviation for the PCA features over the last 20 packets
# .fillna(0) handles the first 20 rows that won't have enough data
rolling_std = pd.DataFrame(X_amp).rolling(window = 20).std().fillna(0).values

# combines the amplitude (a static feature) with delta (a dynamic feature) and rolling_std (a dynamic feature for small movements) into a large table
X_final = np.hstack([X_amp, delta, rolling_std])
print(f"Training the model...")

# here we will split the data into two parts:
# X_train, y_train (95%) is used to train the model
# X_test, y_test (5%) is used to test the model
X_train, X_test, y_train, y_test = train_test_split(X_final, y_raw, test_size = 0.05)

# we create a 'RandomForest' which consists of 50 decision trees (n_estimators = 50)
model = RandomForestClassifier(n_estimators = 50)

# the model looks at X_train (CSI patterns) and learns to associate them with y_train (labels)
model.fit(X_train, y_train)

# we ask the model to predict the labels for the test data (X_test)
# the model will not see the real answers (y_test) just yet
y_pred = model.predict(X_test)

# we compare the model's guesses (y_pred) with the real answers (y_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the model: {accuracy * 100:.2f}%")

ser = None # initialize the serial connection variable as 'None' (empty)

try:       # the 'try' block let's us test a block of code for errors
    # Initialize Serial Connection (opens the specific serial port at the specified speed, 'timeout = 0.1' means it will wait 100ms for data before moving on)
    ser = serial.Serial(COM_PORT, BAUD_RATE, timeout = 0.1)
    
    ser.reset_input_buffer()   # clear the buffer to get rid of old data
    
    probability_buffer = []    # list to save recent probability predictions for optimizations
    buffer_size = 10           # number of recent predictions to average (determines reaction speed)

    min_threshold = 0.70       # decision threshold (min)     
    max_threshold = 0.85       # decision threshold (max) 
    person_detected = False    # defines a state variable outside the loop

    previous_pca = None        # variable store previous PCA vector for delta calculation

    SG_buffer = deque(maxlen = 11)  # buffer for Savitzky-Golay. It is used to store the last 11 packets to apply the filter for live data
    std_buffer = deque(maxlen = 20) # buffer for rolling standard deviation. It is used to store the last 20 PCA features and looks for movements

    print(f"Collecting data for calibration...")

    # we collect data when the room is empty for like 30 seconds (we do that, so our model is able to predict if the room is empty or not regardless of the objects/furniture in the room and if they changed position since we collected data for training)
    calibration_data = []

    # we save the time the calibration process started
    start_calibration_time = time.time()

    # this gives us enough time to leave the room
    time.sleep(20)

    # we collect data for 30 seconds to learn the "empty room" state
    while time.time() - start_calibration_time < 30:
        try: 
            raw_line = ser.readline()
            if not raw_line: continue

            line = raw_line.decode('utf-8', errors = 'ignore').strip()

            if line.startswith("[CSI DATA]"):
                numbers_found = re.findall(r'-?\d+', line)

                if len(numbers_found) > 3 and numbers_found[2] == '384':
                    csi_raw = [int(x) for x in numbers_found[-384:]]

                    if len(csi_raw) >= 382:
                        csi_np = np.array(csi_raw)

                        real_part = csi_np[0::2]
                        imaginary_part = csi_np[1::2]
      
                        ampl = np.sqrt(real_part**2 + imaginary_part**2)
                        
                        if ampl.shape[0] > X_mean.shape[0]:
                            ampl = ampl[:X_mean.shape[0]]

                        ampl = (ampl - X_mean) / (X_std + 0.00001)

                        # store this "empty room" packet in our list
                        calibration_data.append(ampl)
        except:
            continue

    # check if the list is empty        
    if len(calibration_data) > 0:
        # calculate the average of the "empty room" signals (axis = 0 -> rows)
        # this "baseline_static" represents the furniture, walls, and static environment
        baseline_static = np.mean(calibration_data, axis = 0)
    else:
        baseline_static = 0

    while True:
        try:
            ser.reset_input_buffer()   # clear the serial buffer to get rid of old data
            raw_line = ser.readline()  # read the most recent line of data available

            if not raw_line: continue
            
            # decodes the bytes to text (utf-8) and gets rid of unwanted spaces (.strip())
            # errors = 'ignore' is used to ignore corrupted characters
            line = raw_line.decode('utf-8', errors = 'ignore').strip()

            # check if the row starts with the key text we set
            if line.startswith("[CSI DATA]"):
                # re.findall(r'-?\d+',line) scans the entire line of text
                # r'-?\d+' -> searches for any group of digits (\d+)
                numbers_found = re.findall(r'-?\d+', line)

                # we check if the length of the CSI string is 384, if it is, we keep it
                if len(numbers_found) > 3 and numbers_found[2] == '384':
                    # [-384] -> we grab the last 384 numbers in the line (where the important data is)
                    csi_raw = [int(x) for x in numbers_found[-384:]]

                    # ignores the bad lines thus filtering the data
                    if len(csi_raw) >= 382:
                        # converts the Python list into a NumPy array (for time optimization)
                        csi_np = np.array(csi_raw)

                        # the raw data is interleaved (real, imaginary, real, imaginary...)
                        # we extract the real parts by taking every 2nd column starting from index 0
                        # we extract the imaginary parts by taking every 2nd column starting from index 1
                        real_part = csi_np[0::2]
                        imaginary_part = csi_np[1::2]
                        
                        # we calculate the amplitude (signal strength) using Pythagorean theorem
                        # sqrt(real^2 + imaginary^2)
                        ampl = np.sqrt(real_part**2 + imaginary_part**2)
                        
                        # ensures the live packet has the same number of columns as the training data
                        if ampl.shape[0] > X_mean.shape[0]:
                            ampl = ampl[:X_mean.shape[0]]

                        # standardize using the mean and std learned during training
                        ampl = (ampl - X_mean) / (X_std + 0.00001)

                        # we subtract the baseline (furniture, walls, objects, etc.) from the current signal 
                        # if room is empty, this value is approximately 0. If room is not empty, this value is greater than 0.
                        ampl = ampl - baseline_static

                        # add amplitude to rolling buffer for smoothing 
                        SG_buffer.append(ampl)

                        # wait until we have enough packets in our buffer to filter
                        if len(SG_buffer) < 11: continue
                            
                        # converts buffer to 2D array
                        batch_of_data = np.array(SG_buffer)  

                        # apply filter to the previously converted array
                        batch_of_smoothed_data = savgol_filter(batch_of_data, window_length = 11, polyorder = 3, axis = 0)

                        # takes the most recent smoothed packet (last row)
                        current_smoothed_data = batch_of_smoothed_data[-1].reshape(1,-1)  

                        current_pca = pca.transform(current_smoothed_data)

                        if previous_pca is None:
                            delta = np.zeros_like(current_pca)          # since we have no history, for the first packet delta will be 0
                        else:
                            delta = np.abs(current_pca - previous_pca)  # calculate the absolute difference between curremt packet and the previous one
                        
                        previous_pca = current_pca                      # update history

                        # we append the flattened array to the deque so we can calculate standard deviation easier
                        std_buffer.append(current_pca.flatten())

                        # wait for std_buffer to fill with 20 packets
                        if len(std_buffer) < 20: continue

                        # calculate standard deviation of the buffer across time (axis = 0)
                        current_std = np.std(np.array(std_buffer), axis = 0).reshape(1,-1)

                        # combine pca and delta to match training data format
                        features = np.hstack([current_pca, delta, current_std])

                        # reshape to (1, n_features) to make it a 2D array (1 is the number of rows, -1 means that the number of columns is unknown)
                        features_ready = features.reshape(1,-1)

                        # get the probability that the room is not empty
                        probability = model.predict_proba(features_ready)[0][1]
                        
                        # adding the probability to the circular buffer
                        probability_buffer.append(probability)
                        if len(probability_buffer) > buffer_size:
                            probability_buffer.pop(0) # remove the oldest value to maintain buffer size

                        # calculate the median score (good filter)
                        confidence = np.median(probability_buffer)  

                        # hysteresis logic for more accurate predictions
                        if person_detected:
                            # switch to 'empty' if the score drops below our threshold
                            if confidence < min_threshold:
                                person_detected = False
                                info = "empty    "
                            else:
                                info = "not empty"     
                        else:
                            # switch to 'not empty' if score goes above our threshold           
                            if confidence > max_threshold:
                                person_detected = True
                                info = "not empty"
                            else:
                                info = "empty    "
                        
                        print(f"RSSI: {numbers_found[1]}dBm, CONFIDENCE: {confidence:.4f}, STATUS: {info}", end = '\r')    
        except (ValueError, IndexError):
            continue

# the 'except' block is used for error handling outside the loop         
except serial.SerialException as e:
    print(f"Error: {e}")                      # if the port is busy or the cable is unplugged
except KeyboardInterrupt:
    print(f"Testing interrupted by User.")    # if we press 'Interrupt' in Jupyter

# the 'finally' block will be executed regardless if the try block raises an error or not
finally:
    if ser is not None and ser.is_open: # check if the serial connection was opened
        ser.close()                     # if it was opened, we will close the USB port

Loading and processing the dataset...
Training the model...
Accuracy of the model: 99.48%
Testing interrupted by User. STATUS: not empty
